In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd
import requests
import csv
import kaggle 
import pickle
from joblib import dump, load

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn import preprocessing 


# Modelling Helpers
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import  Normalizer , scale
from sklearn.model_selection import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import auc, roc_curve, roc_auc_score
# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

In [2]:
#kaggle.api.authenticate()
#kaggle.api.dataset_download_files('glebsolomennikov/main-task', path='data/', unzip=True)

In [3]:
data = pd.read_csv('data/data.csv')
model = pd.read_csv('data/model.csv')

In [4]:
data = data.merge(model, how='inner', left_on = 'incident', right_on = 'incident')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82617 entries, 0 to 82616
Columns: 1911 entries, incident to PD
dtypes: float64(1838), int64(72), object(1)
memory usage: 1.2+ GB


In [6]:
for_pred = data[-1000:]
data = data[:-1000]
data['target'] = abs(data.flg_90_12_add - data.PD)
data = data.replace(np.nan, 0)

In [7]:
del data['flg_90_12_add']
del data['PD']

In [8]:
le = preprocessing.LabelEncoder()

In [9]:
X = data.apply(le.fit_transform)
from sklearn.decomposition import PCA
pca = PCA(n_components = 1)
XPCAreduced = pca.fit_transform(np.transpose(X))

In [10]:
y =X['target']
del X['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train.shape, X_test.shape

((61212, 1909), (20405, 1909))

In [11]:
estimators = [
    ('lr', RidgeCV()),
    ('svr', LinearSVR(random_state=421) )]
linreg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42))

In [12]:
%%time
linreg.fit(X_train, y_train)

Wall time: 7min 15s


StackingRegressor(estimators=[('lr', RidgeCV(alphas=array([ 0.1,  1. , 10. ]))),
                              ('svr', LinearSVR(random_state=421))],
                  final_estimator=RandomForestRegressor(n_estimators=10,
                                                        random_state=42))

In [13]:
for_pred_mod = for_pred.apply(le.transform)
del for_pred_mod['flg_90_12_add']
del for_pred_mod['PD']


ValueError: y contains previously unseen labels: [20264239, 20264362, 20264564, 20266924, 20268570, 20268684, 20269502, 20270248, 20271314, 20272681, 20272839, 20272945, 20273658, 20276555, 20277303, 20277337, 20282461, 20282864, 20283051, 20283238, 20283404, 20284017, 20284370, 20284955, 20285052, 20286298, 20288859, 20290047, 20290596, 20292154, 20293464, 20293899, 20296354, 20296713, 20296714, 20296949, 20300980, 20301523, 20301891, 20302856, 20303170, 20303324, 20304985, 20305297, 20306632, 20307040, 20307310, 20307689, 20307785, 20310375, 20311581, 20311635, 20313051, 20314745, 20315398, 20315821, 20316598, 20318515, 20319470, 20321060, 20322190, 20322655, 20323796, 20323885, 20324008, 20325270, 20327088, 20328803, 20329876, 20330172, 20333464, 20333622, 20338377, 20338457, 20339975, 20345113, 20346420, 20347296, 20347499, 20348084, 20348394, 20348581, 20348797, 20350683, 20352121, 20354900, 20356916, 20358015, 20358028, 20359041, 20359663, 20360080, 20360149, 20360992, 20361111, 

In [ ]:

print('TRAIN MSE:', linreg.score(X_train, y_train))
print('TEST MSE:', linreg.score(X_test, y_test))
res = linreg.predict(for_pred_mod)
prediction = pd.DataFrame(res, columns=['predictions']).to_csv('ans_on_x_y.csv',encoding='utf-8', index=False)

In [ ]:
len(res)

In [ ]:
linreg.fit(X, y)

In [ ]:
print('TRAIN MSE:', linreg.score(X_train, y_train))
print('TEST MSE:', linreg.score(X_test, y_test))

In [ ]:
res = linreg.predict(for_pred_mod)
prediction = pd.DataFrame(res, columns=['predictions']).to_csv('ans_on_X.csv',encoding='utf-8', index=False)